# Introduction to Data Science
## Dealing with data Pythonically

Sophia Maria Tsilerides

***

This is a simple notebook that displays the importance of data exploration taking advantage of the Python programming language. Typically, databases do not collect data in a structured way to meet the need of data scientists. Therefore, it is critical to manipulate and analyze the data to prepare for feature engineering and data leakage. With a better understanding of the data, we can create models that are more tailored to yield better results!

In [1]:
import pandas as pd
import numpy as np
import os

First, we load the data set `"ads_dataset.tsv"` into a Python Pandas data frame called `ads`.

In [2]:
directory = os.getcwd()
data = pd.read_csv(directory + '/ads_dataset.tsv', sep='\t') 
ads = pd.DataFrame(data)
ads.head()

,isbuyer,buy_freq,visit_freq,buy_interval,sv_interval,expected_time_buy,expected_time_visit,last_buy,last_visit,multiple_buy,multiple_visit,uniq_urls,num_checkins,y_buy
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,106,106,0,0,169,2130,0
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,72,72,0,0,154,1100,0
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,5,5,0,0,4,12,0
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,6,6,0,0,150,539,0
NaN,0,NaN,2,0.0,0.5,0.0,-101.1493,101,101,0,1,103,362,0


Next, the Python function `getDfSummary()` present summary statistics in the following way:
- Takes as input a data frame
- For each variable in the data frame calculates the following features:
  - `number_nan` to count the number of missing not-a-number values
  - Ignoring missing, NA, and Null values:
    - `number_distinct` to count the number of distinct values a variable can take on
    - `mean`, `max`, `min`, `std` (standard deviation), and `25%`, `50%`, `75%` to correspond to the appropriate percentiles
- All of these new features should be loaded in a new data frame. Each row of the data frame should be a variable from the input data frame, and the columns should be the new summary features.
- Returns this new data frame containing all of the summary information

This function can help us identify which features need more attention before analysis. 

In [5]:
def getDfSummary(input_data):
    
    output_data = pd.DataFrame(input_data.describe()) #.describe() excludes NaN values - tested in appendix
    output_data = output_data.drop('count')
    output_data = output_data.transpose()
    output_data['number_nan'] = input_data.isna().sum()
    output_data['number_distinct'] = input_data.nunique() #excludes NaN values - tested in appendix
    return output_data

new_ads = getDfSummary(ads)
new_ads.head()

,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,0.042632,0.202027,0.0,0.0,0.0,0.000000,1.0000,0,2
buy_freq,1.240653,0.782228,1.0,1.0,1.0,1.000000,15.0000,52257,10
visit_freq,1.852777,2.921820,0.0,1.0,1.0,2.000000,84.0000,0,64
buy_interval,0.210008,3.922016,0.0,0.0,0.0,0.000000,174.6250,0,295
sv_interval,5.825610,17.595442,0.0,0.0,0.0,0.104167,184.9167,0,5886


For resource management let's see how long does it take the `getDfSummary()` function to work. %timeit is a Python magic command that runs the function many times and return the average time it took. Looks like the summary function takes 86.3 ms on average to run.

In [6]:
%timeit getDfSummary(ads)

86.3 ms ± 986 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


We specifically see that `buy_freq` has missing values. What does this tell us?

In [7]:
new_ads[new_ads['number_nan'] > 0]

,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
buy_freq,1.240653,0.782228,1.0,1.0,1.0,1.0,15.0,52257,10


It does not look like the data is missing at random. If the buyer is 0 then the "buy_freq" is NaN. Assuming the data is meaningful, there are 52,257 instances in this dataset where there were transactions done by people who were not buyers. There could instead be sellers, retailers, wholesalers, etc. that are generating this data. In this case, they wouldn't be purchasing anything, hence "buy_freq" being NaN, however, there is a lot of other information about these parties. It must be meaningful since the mean and standard deviation of the “non-buyers” data points parallel the data points of buyers. For example, the standard deviation of "expected_time_visit" between the two partied have a difference of 11 and the mean of “num_checkins” have a difference of 23. It would also appear that the dataset is reasonably split between "non buyers" and buyers with 207 unique urls for “non-buyers” and 196 urls for buyers. Since they are a significant part of the sample, it is worth understanding them better and not discarding the data. 

If with more research the type of customer these users are becomes clear, they can instead be classified as Type 2: Seller, for example, and given their own unique metric “seller_freq” as opposed to “buy_freq,” which can be calculated through the other given variables. 


In [8]:
bf_nulls = ads[ads['buy_freq'].isnull()]
getDfSummary(bf_nulls)

,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1
buy_freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52257,0
visit_freq,1.651549,2.147955,1.0000,1.0,1.0,2.000000,84.00000,0,48
buy_interval,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1
sv_interval,5.686388,17.623555,0.0000,0.0,0.0,0.041667,184.91670,0,5112
expected_time_buy,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1
expected_time_visit,-9.669298,31.239030,-187.6156,0.0,0.0,0.000000,91.40192,0,13351
last_buy,65.741317,53.484622,0.0000,19.0,52.0,106.000000,188.00000,0,189
last_visit,65.741317,53.484622,0.0000,19.0,52.0,106.000000,188.00000,0,189
multiple_buy,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.00000,0,1


In [9]:
bf_nulls.head()

,isbuyer,buy_freq,visit_freq,buy_interval,sv_interval,expected_time_buy,expected_time_visit,last_buy,last_visit,multiple_buy,multiple_visit,uniq_urls,num_checkins,y_buy
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,106,106,0,0,169,2130,0
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,72,72,0,0,154,1100,0
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,5,5,0,0,4,12,0
NaN,0,NaN,1,0.0,0.0,0.0,0.0000,6,6,0,0,150,539,0
NaN,0,NaN,2,0.0,0.5,0.0,-101.1493,101,101,0,1,103,362,0


In [10]:
bf = ads[ads['buy_freq'].notnull()]
getDfSummary(bf)

,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,1.000000,0.000000,1.0000,1.000000,1.000000,1.00000,1.00000,0,1
buy_freq,1.240653,0.782228,1.0000,1.000000,1.000000,1.00000,15.00000,0,10
visit_freq,6.371723,8.680026,0.0000,2.000000,3.000000,7.00000,75.00000,0,60
buy_interval,4.926110,18.377272,0.0000,0.000000,0.000000,0.00000,174.62500,0,295
sv_interval,8.952092,16.651559,0.0000,0.041667,3.354167,10.83333,176.70830,0,1367
expected_time_buy,-4.645390,23.779721,-181.9238,0.000000,0.000000,0.00000,84.28571,0,348
expected_time_visit,-22.370870,42.080323,-183.3626,-34.575625,-1.504768,0.00000,88.30038,0,1788
last_buy,42.003438,47.980703,0.0000,3.000000,21.000000,70.00000,186.00000,0,186
last_visit,42.003438,47.980703,0.0000,3.000000,21.000000,70.00000,186.00000,0,186
multiple_buy,0.149119,0.356282,0.0000,0.000000,0.000000,0.00000,1.00000,0,2


In [11]:
bf.head()

,isbuyer,buy_freq,visit_freq,buy_interval,sv_interval,expected_time_buy,expected_time_visit,last_buy,last_visit,multiple_buy,multiple_visit,uniq_urls,num_checkins,y_buy
NaN,1,1.0,2,0.0,0.958333,0.0,-20.671200,21,21,0,1,65,175,0
NaN,1,1.0,3,0.0,0.437500,0.0,-0.770371,1,1,0,1,101,395,0
NaN,1,1.0,2,0.0,1.083333,0.0,-34.036930,34,34,0,1,0,1775,0
NaN,1,1.0,4,0.0,9.486111,0.0,3.196425,3,3,0,1,148,2918,0
NaN,1,1.0,1,0.0,0.000000,0.0,0.000000,79,79,0,0,5,118,0


In [12]:
new_ads[new_ads['number_distinct'] == 2]

,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,0.042632,0.202027,0.0,0.0,0.0,0.0,1.0,0,2
multiple_buy,0.006357,0.079479,0.0,0.0,0.0,0.0,1.0,0,2
multiple_visit,0.277444,0.447742,0.0,0.0,0.0,1.0,1.0,0,2
y_buy,0.004635,0.067924,0.0,0.0,0.0,0.0,1.0,0,2
